In [ ]:
import os
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

In [ ]:
# Directories of image folders
fake_folder = "fake_images/fake"
real_folder = "img_align_celeba/real"

# Hyperparameters
batch_size = 256
epochs = 10
input_shape = (128, 128, 3)

# Lists of image files
fake_files = [os.path.join(fake_folder, file) for file in os.listdir(fake_folder)]
real_files = [os.path.join(real_folder, file) for file in os.listdir(real_folder)]

# Randomly select 1000 fake and 1000 real images
random.seed(42)  # Set a random seed for reproducibility
selected_fake_files = random.sample(fake_files, 1000)
selected_real_files = random.sample(real_files, 1000)

# Combine the lists of selected files and labels (0 for fake and 1 for real)
selected_files = selected_fake_files + selected_real_files
selected_labels = [0] * 1000 + [1] * 1000

# Convert labels to NumPy arrays
selected_labels = np.array(selected_labels)

# Create a pandas DataFrame with 'File' and 'Label' columns
df = pd.DataFrame({'File': selected_files, 'Label': selected_labels})

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.1)

train_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='File',
    y_col='Label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='raw',  # Changed to 'raw' to work with numerical labels
    subset='training'
)

validation_generator = train_datagen.flow_from_dataframe(
    df,
    x_col='File',
    y_col='Label',
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode='raw',  # Changed to 'raw' to work with numerical labels
    subset='validation'
)

# CNN architecture definition
model = keras.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Model compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Model training
model.fit(train_generator, epochs=epochs, validation_data=validation_generator)

In [ ]:
# Model evaluation
loss, accuracy = model.evaluate(validation_generator)
print(f'Accuracy on the validation set: {accuracy * 100:.2f}%')

In [ ]:
# Saving the trained model
model.save('classification_real_fake_model.h5')